<a href="https://colab.research.google.com/github/irajamuller/data_science/blob/main/Engenharia_de_Atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Engenharia de Atributos**
---

##1.1-Codificação de Dados
<p align="justify">
Este notebook irá explorar a utilização do Scikit-Learn para a codificação de variáveis categóricas em conjuntos de dados. A maioria dos algoritmos de aprendizado de máquina é compatível apenas com atributos numéricos, não sendo possível a utilização de variáveis categóricas sem algum tipo de preparação. Por isso, é comum o uso de métodos de codificação para tornar variáveis categóricas em algum tipo de variável numérica.
</p>

<p align="justify">
Iremos explorar dois tipos de codificação neste notebook: one-hot e por etiquetas. Ambos tipos de codificação são implementados como técnicas de pré-processamento disponíveis na biblioteca Scikit-Learn. Iremos utilizar ambas as técnicas para realizar a codificação de atributos categóricos do conjunto de dados `tips` disponível na biblioteca Seaborn. Primeiramente iremos importar as bibliotecas Pandas e Seaborn para manipular o conjunto de dados de exemplo.
</p>

In [ ]:
import pandas as pd
import seaborn as sns

<p align="justify">
O método one-hot será o primeiro que iremos trabalhar. Este método de codificação converte dados categóricos em uma forma numérica que possa ser usada por algoritmos de aprendizado de máquina. Cada categoria única de uma variável se torna um atributo binário onde 0 indica a ausência da categoria e 1 indica a presença.
</p>

<p align="justify">
Por exemplo, suponha que o conjunto de dados possua uma categoria "gênero" que classifica indivíduos como masculino ou feminino. Antes do processo de codificação, um exemplo de conjunto de dados seria:
</p>

<center>

| Nome  | Gênero |
|-------|--------|
| João  | M      |
| Maria | F      |

</center>

Após o processo de codificação, teríamos o seguinte conjunto de dados:

<center>

| Nome  | Gênero_M | Gênero_F |
|-------|----------|----------|
| João  | 1        | 0        |
| Maria | 0        | 1        |

</center>

A célula abaixo apresenta um exemplo do uso de codificação one-hot sobre o conjunto de dados <strong>tips</strong>.

In [ ]:
# Importar a classe OneHotEncoder do Scikit-Learn
from sklearn.preprocessing import OneHotEncoder

# Carregar o conjunto de dados "tips" através do Seaborn
dataset = sns.load_dataset('tips')

# Iremos criar um subconjunto de dados contendo apenas as colunas que irão passar pelo processo de codificação one-hot
categorias = dataset[['sex', 'smoker', 'day', 'time']]

#  Agora criamos a instância da classe OneHotEncoder e a utilizamos para criar a codificação dos atributos
encoder = OneHotEncoder(sparse_output=False) # sparse_output (denso/esparso)
categorias_encoded = encoder.fit_transform(categorias)

# Após o processo de codificação, precisamos criar de forma explícita um dataframe contendo o resultado do processo
feature_names = encoder.get_feature_names_out(categorias.columns)
categorias_encoded_df = pd.DataFrame(categorias_encoded, columns=feature_names)

# Por fim, podemos substituir os dados originais pelos novos atributos criados através da codificação.
# Precisamos primeiramente "alinhar" os índices do conjunto de dados com a codificação one-hot com
# aqueles do conjunto original. Em seguida, podemos remover as colunas com os atributos originais
# pelos atributos criados pela codificação.
categorias_encoded_df.index = dataset.index
dataset = dataset.drop(columns=categorias.columns)
dataset = pd.concat([dataset, categorias_encoded_df], axis=1)

dataset.head()

<p align="justify">
O próximo método com o qual iremos trabalhar é a codificação por etiquetas ou <strong>label encoding</strong>. Este método transforma dados categóricos em valores numéricos inteiros. Por exemplo, suponha que o conjunto de dados possua uma categoria "gênero" que classifica indivíduos como masculino ou feminino. Antes do processo de codificação, um exemplo de conjunto de dados seria:
</p>

<center>

| Nome  | Gênero |
|-------|--------|
| João  | M      |
| Maria | F      |

</center>

Após o processo de codificação, teríamos o seguinte conjunto de dados:

<center>

| Nome  | Gênero |
|-------|--------|
| João  | 1      |
| Maria | 2      |

</center>

<p align="justify">
A codificação por etiquetas está disponível na classe <strong>LabelEncoder</strong> do Scikit-Learn. A célula abaixo apresenta um exemplo do uso deste tipo de codificação nos atributos do conjunto de dados <strong>tips</strong>.
</p>

In [ ]:
# Importar a classe LabelEncoder do Scikit-Learn
from sklearn.preprocessing import LabelEncoder

# Carregar o conjunto de dados "tips" através do Seaborn
dataset = sns.load_dataset('tips')

# Neste exemplo precisamos apenas criar uma lista com os atributos que passarão pelo processo de codificação
categorias = ['sex', 'day']

# Vamos criar uma instância do LabelEncoder para realizar o processo de codificação
label_encoder = LabelEncoder()

# Iteramos pela lista de atributos que passarão pela codificação. Para cada atributo, criamos um novo
# atributo no conjunto de dados, contendo a respectiva versão codificada.
for col in categorias:
    # Codificar os valores da coluna
    dataset[f"{col}_encoded"] = label_encoder.fit_transform(dataset[col])

dataset.head()

##1.2-Seleção de Atributos Categóricos - SelectKBest
<p align="justify">
A seleção de atributos se trata de um conjunto de técnicas que permite identificar quais os atributos apresentam maior relevância para a predição de uma determinada variável alvo. Estes métodos envolvem técnicas estatísticas que analisam diferentes características do conjunto de dados e buscam estabelecer correlações entre os valores de diferentes atributos.
</p>

<p align="justify">
A classe <strong>SelectKBest</strong> do Scikit-Learn implementa um processo de análise e seleção dos atributos de maior relevância em um conjunto de dados. Ela permite o uso de diferentes métodos estatísticos para análises de correlação, tais como o qui-quadrado ou o ANOVA. Cada um destes testes pode ser aplicado em diferentes contextos. Por exemplo, o teste do qui-quadrado pode ser aplicado para identificar a relevância de variáveis categóricas, mas não terá resultados aceitáveis no caso de variáveis contínuas.
</p>

<p align="justify">
Este notebook apresenta um exemplo da aplicação do teste de qui-quadrado através da classe <strong>SelectKBest</strong> em um subconjunto do dataset <strong>titanic</strong>, disponível através do Seaborn. Nosso objetivo é identificar qual das variáveis categóricas presentes nesse conjunto de dados apresenta maior relevância para a classificação da variável alvo <strong>survivor</strong>.
</p>

<p align="justify">
Primeiramente iremos carregar as bibliotecas que utilizaremos ao longo do nosso experimento. Além do Pandas e do Seaborn, que serão utilizados para carregar e manipular o conjunto de dados, também carregaremos módulos específicos da Scikit-Learn para uso da seleção de atributos.
</p>

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder

<p align="justify">
O próximo passo será carregar o conjunto de dados e realizar alguns tratamentos iniciais para permitir que a seleção de atributos seja executada. Em suma:
</p>

- Remoção das variáveis contínuas como <strong>age</strong> e <strong>fare</strong>;
- Remoção das entradas com valores nulos;
- Codificação das variáveis categóricas através do <strong>LabelEncoder</strong>.

In [ ]:
# Carregar conjunto de dados Titanic
titanic = sns.load_dataset("titanic")

# Pré-processamento:
# - Selecionar colunas relevantes
# - Remover valores ausentes
titanic = titanic[['survived', 'pclass', 'sex', 'embarked']]
titanic = titanic.dropna()

# Codificar variáveis categóricas para numéricas
encoder = LabelEncoder()
titanic['sex'] = encoder.fit_transform(titanic['sex'])
titanic['embarked'] = encoder.fit_transform(titanic['embarked'])

<p align="justify">
O próximo passo será dividir o conjunto de dados entre as variáveis preditoras (usadas para descoberta da classe) e a variável alvo (aquela que se deseja descobrir a classe).
</p>

In [ ]:
# Separar variáveis preditoras (X) e alvo (y)
X = titanic[['pclass', 'sex', 'embarked']]
y = titanic['survived']

<p align="justify">
A próxima célula instancia a classe `SelectKBest`, definindo o qui-quadrado (`chi2`) como teste estatístico a ser utilizado. Após a aplicação do teste, são extraídos os resultados do teste, os quais são inseridos em um Dataframe para análise ao final da célula. Os resultados obtidos com o teste possuem, para cada atributo, os seguintes valores:
</p>

- <strong>chi2 score</strong>: indica o quanto os valores observados de uma variável diferem dos valores esperados sob a hipótese nula. Valores mais altos neste resultado sugerem uma associação mais forte entre o atributo e a variável alvo.
- <strong>p-value</strong>: é a probabilidade de observar um Chi2-Score tão extremo quanto o calculado assumindo que a hipótese nula seja verdadeira. Um p-value menor que 0,05 sugere que há associação entre o atributo e a variável e, portanto, o atributo é relevante para a predição.

In [ ]:
# Seleção de atributos com o teste qui-quadrado
selector = SelectKBest(score_func=chi2, k='all')
X_selected = selector.fit_transform(X, y)

# Obter pontuações do qui-quadrado e p-valores
chi2_scores = selector.scores_
chi2_pvalues = selector.pvalues_

# Exibir resultados em um DataFrame
df_chi2 = pd.DataFrame({
    'Feature': X.columns,
    'Chi2-Score': chi2_scores,
    'p-Value': chi2_pvalues
}).sort_values(by='Chi2-Score', ascending=False)

print("Resultados do Teste Qui-Quadrado:")
df_chi2

##1.3-Seleção de Atributos Categóricos - RFE
<p align="justify">
A eliminação recursiva de atributos (RFE ou <strong>random feature elimination</strong>) é outro método que podemos utilizar para identificar os atributos de maior relevância em um conjunto de dados. Ele funciona de maneira iterativa, treinando um modelo base e avaliando a importância de cada atributo. Em cada etapa, o atributo menos relevante é removido, e o modelo é re-treinado com os atributos restantes. Esse processo continua até que o número desejado de atributos seja alcançado. O RFE é útil para reduzir a dimensionalidade do conjunto de dados, melhorar a eficiência do modelo e minimizar o risco de overfitting, especialmente em problemas com muitos atributos redundantes ou irrelevantes.
</p>

<p align="justify">
Conforme mencionado, a operação do RFE requer o uso de um algoritmo de aprendizado de máquina mais simples, tal como uma árvore de decisão ou uma random forest. Modelos mais complexos também podem ser utilizados, mas sua complexidade pode afetar o tempo necessário para a obtenção do resultado.
</p>

<p align="justify">
Neste notebook iremos utiliza o método de RFE disponibilizado pela biblioteca Scikit-Learn em conjunto com um algoritmo de árvore de decisão. O processo será aplicado sobre o conjunto de dados `titanic`. Primeiramente iremos importar as bibliotecas Pandas e Seaborn para que possamos carregar e manipular o conjunto de dados. Também iremos carregar os módulos necessários do Scikit-Learn para a utilização do método de RFE.
</p>

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder

<p align="justify">
A célula a seguir realiza o carregamento e preparação do conjunto de dados. Os seguintes passos serão executados:
</p>

- Carregamento do conjunto de dados;
- Seleção dos atributos de interesse dentro do conjunto
- Eliminação dos registros com valores faltantes;
- Codificação em etiquetas dos atributos categóricos <strong>sex</strong> e <strong>embarked</strong>.

In [ ]:
# Carregar o dataset Titanic
titanic = sns.load_dataset("titanic")

# Selecionar colunas relevantes e tratar valores ausentes
titanic = titanic[['survived', 'pclass', 'sex', 'age', 'fare', 'embarked']]
titanic = titanic.dropna()

# Codificar variáveis categóricas
encoder = LabelEncoder()
titanic['sex'] = encoder.fit_transform(titanic['sex'])
titanic['embarked'] = encoder.fit_transform(titanic['embarked'])

<p align="justify">
Assim como no uso do SelectKBest, para utilizar o RFE precisamos dividir o conjunto de dados entre os atributos preditores e o atributo alvo. A célula a seguir realiza esta divisão.
</p>

In [ ]:
# Separar variáveis preditoras (X) e alvo (y)
X = titanic[['pclass', 'sex', 'age', 'fare', 'embarked']]
y = titanic['survived']

<p align="justify">
Após a preparação do conjunto de dados, podemos aplicar o método de RFE utilizando a árvore de decisão. A célula abaixo instancia as classes necessárias para aplicar o método RFE e, em seguida, prepara um Dataframe para exibição dos resultados do processamento.
</p>

In [ ]:
# Primeiramente iremos criar uma instância de um modelo de árvore de decisão para utilizar como base do RFE.
model = DecisionTreeClassifier()

# Em seguida, criamos a instância do RFE propriamente dito, utilizando como
# modelo base a árvore de decisão. Também definimos que queremos que o processo
# seja encerrado quando tivermos encontrado os três atributos de mairo relevância.
selector = RFE(estimator=model, n_features_to_select=3)
X_selected = selector.fit_transform(X, y)

# Após o processamento, podemos criar um dataframe para exibir os resultados do processamento
df_rfe = pd.DataFrame({
    'Feature': X.columns,
    'Selected': selector.support_,  # True para os atributos selecionados
    'Ranking': selector.ranking_    # 1 para os mais relevantes
}).sort_values(by='Ranking')

print("Resultados do RFE:")
df_rfe

## 1.4-Vetorização de Textos

<p align="justify">
Métodos de aprendizado de máquina voltados para processamento de linguagem natural requerem a utilização de métodos que permitam a representação numérica de textos e suas relações. Um dos métodos mais comuns para a representação de textos é o <strong>Term Frequency-Inverse Document Frequency</strong> ou TF-IDF, uma técnica que combina duas métricas: a frequência de um termo em um documento (TF) e a relevância desse termo em relação ao conjunto de documentos (IDF). ssa abordagem transforma documentos textuais em vetores numéricos, atribuindo pesos maiores a palavras que aparecem frequentemente em um documento, mas raramente nos outros, ajudando a identificar termos importantes e distintivos.
</p>

<p align="justify">
O TF-IDF é amplamente utilizado em tarefas de processamento de linguagem natural, como classificação de texto, análise de sentimentos e recuperação de informações. Ele é especialmente útil em conjuntos de dados como artigos de notícias, e-mails (para classificação de spam), resenhas de produtos, ou qualquer coleção de textos onde seja importante destacar as palavras mais representativas de cada documento sem que palavras muito comuns (como "e", "de", "o") dominem a análise.
</p>

<p align="justify">
Este notebook apresenta um exemplo prático da utilização do método TF-IDF para a vetorização de um conjunto de dados com algumas frases em português. O método encontra-se disponível na biblioteca Scikit-Learn através da classe <strong>TfidfVectorizer</strong>, a qual será carregada na célula abaixo. Também carregaremos outros módulos necessários para utilizar o TF-IDF no Scikit-Learn, além da biblioteca Pandas para criar e gerenciar o conjunto de dados de exemplo.
</p>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

<p align="justify">
Iremos trabalhar com conjuntos de dados contido no arquivo <strong>spam.csv</strong>. Este arquivo contém amostras de frases que são classificadas como sendo de mensagens indesejadas (spam) ou não.
</p>

In [ ]:
# Realiza a leitura do conjunto de dados para um Dataframe
messages = pd.read_csv('https://raw.githubusercontent.com/irajamuller/data_science/main/dataset/spam.csv')

messages.head()

<p align="justify">
Neste exemplo iremos trabalhar com o treinamento de um modelo de aprendizado baseado em Random Forest. Faremos isso apenas como exemplo da aplicação do conjunto de dados processado através do TF-IDF. Por isso, a célula a seguir divide os atributos do conjunto de dados entre preditores (message) e alvo (is_spam).
</p>

In [ ]:
# Divide os atributos entre preditores (X) e alvo (y).
X = messages['message']
y = messages['is_spam']

<p align="justify">
A próxima etapa será aplicar o método TF-IDF em si para realizar a análise das strings contidas no conjunto de dados e criar o vetor de características que poderá ser utilizado em algoritmos de aprendizado de máquina.
</p>

In [ ]:
# Cria uma instância da classe "TfidfVectorizer" para codificação do conjunto de dados
vectorizer = TfidfVectorizer()
# Aplica o processo de análise e codificação sobre o array
X_tfidf = vectorizer.fit_transform(X)

<p align="justify">
Podemos visualizar o resultado do processo de codificação extraindo as informações do resultado da codificação e da instância do codificador. A célula a seguir compila estes resultados e apresenta um sumário com as frequências identificadas para cada uma das palavras presentes no conjunto de dados.
</p>

In [ ]:
# Extrai as frequências obtidas para cada um dos termos que o codificador identificou durante o processo
word_frequencies = X_tfidf.toarray().sum(axis=0)
# Extra o array com todos os termos identificados pelo codificador
vocabulary = vectorizer.get_feature_names_out()

# Cria um Dataframe combinando as informações de vocabulário e frequência e o ordena pela frequência
freq_df = pd.DataFrame({
    'Palavra': vocabulary,
    'Frequência': word_frequencies
})
freq_df.sort_values(by='Frequência', ascending=False)

<p align="justify">
Com o processo de codificação realizado, podemos trabalhar com os dados em diferentes algoritmos de aprendizado. A célula abaixo apresenta um exemplo onde utilizamos o conjunto de dados criado para treinar um modelo simples de Random Forest para o reconhecimento de mensagens de SPAM.
</p>

In [ ]:
# Iremos criar uma divisão do conjunto de dados em partes para treino e teste para realizar
# um processo de treinamento de um modelo de aprendizado de máquina simples.
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3)

# Treinar um modelo Random Forest com os embeddings
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Fazer previsões e avaliar
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Acurácia do modelo com embeddings TF-IDF: {accuracy:.2f}")

## 1.5-Análise de Componente Principal

<p align="justify">
A Análise de Componentes Principais (PCA) é uma técnica estatística utilizada para reduzir a dimensionalidade de conjuntos de dados, transformando variáveis correlacionadas em um novo conjunto de variáveis não correlacionadas chamadas componentes principais. O objetivo do PCA é reduzir a complexidade de conjuntos de dados com muitos atributos, especialmente quando há colinearidade ou redundância entre as variáveis. O PCA alcança isso ao transformar as variáveis originais em um conjunto menor de componentes principais, que mantêm a maior parte da variância dos dados originais.
</p>

<p align="justify">
Um exemplo de um conjunto de dados que pode se beneficiar da redução de dimensionalidade é o dataset <strong>digits</strong>. Este conjunto de dados contém 1797 amostras de dígitos escritos à mão, compostos por imagens em tons de cinza com $8 \times 8$ pixels. Portanto, o conjunto de dados possui um total de 64 atributos, um para cada píxel da imagem. A existência de tantos atributos pode dificultar tarefas como a visualização de dados ou o tratamento do conjunto de dados por algoritmos de aprendizado.
</p>

<p align="justify">
O processo de PCA é adequado para estas situações, tendo como objetivo reduzir a dimensionalidade do conjunto de dados sem ocasionar a perda de informações presentes nos atributos. Neste notebook iremos utilizar o método do PCA para reduzir a dimensionalidade do conjunto de dados digits.
</p>

<p align="justify">
Primeiramente iremos carregar as bibliotecas necessárias para manipular o conjunto de dados e realizar o processo de PCA. Também iremos carregar o Matplotlib para realizar uma análise visual do conjunto de dados.
</p>

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

<p align="justify">
A célula a seguir realiza o carregamento do conjunto de dados digits através da biblioteca Scikit-Learn. Em seguida, aplica-se o método <strong>train_test_split</strong> para dividir o conjunto de dados em partições para treino e teste. Esta divisão de dados é comumente realizada quando trabalhamos com métodos de aprendizado de máquina. No caso do exemplo, a divisão irá destinar 30% do conjunto de dados para testar o modelo de aprendizado criado com os 70% restante do conjunto.
</p>

In [ ]:
# Carregar o dataset Digits do sklearn
data = load_digits()
X, y = data.data, data.target # data é a imagem e target é o dígito

# Divisão treino/teste
# X é o conjunto de dados com as features
# Y é o vetor com os labels ou variávei alvo (o que se deseja prever)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Exibir as primeiras 10 imagens com seus rótulos
fig, axes = plt.subplots(2, 5, figsize=(10, 5))

for i, ax in enumerate(axes.flat):
    ax.imshow(data.images[i], cmap='gray')  # digits.images já tem o formato 8x8
    ax.set_title(f"Label: {digits.target[i]}")
    ax.axis('off')  # tirar os eixos para ficar mais limpo

plt.tight_layout()
plt.show()

<p align="justify">
A célula abaixo cria uma instância da classe PCA disponível no Scikit-Learn. O parâmetro <strong>n_components</strong> define o número de componentes que será criado para o novo dataset. Nosso exemplo transformará os 64 atributos originais em 2 atributos, buscando manter o máximo de informação estatística possível sobre o conjunto nestes atributos. É possível especificar um número maior de atributos, permitindo reter ainda mais informações sobre o conjunto original. Entretanto, um maior número de atributos também irá resultar em maior complexidade computacional quando aplicado a algoritmos de aprendizado.
</p>

In [ ]:
# Análise de Componentes Principais (PCA)
# Reduzir para 2 componentes principais para visualização
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

<p align="justify">
Quando aplicamos o método PCA, cada um dos atributos criados reterá uma parte da informação sobre a variância existente no conjunto de dados original. Podemos verificar o total de variância que foi preservada pelo processo de PCA utilizando o método <strong>explained_variance_ratio_</strong>. Ele apresentará um vetor com o total de variância preservado em cada um dos atributos. Conforme já mencionado, quanto maior o número de atributos, maior será a variância preservada do conjunto original.
</p>

In [ ]:
# Visualizar a variação explicada
explained_variance = pca.explained_variance_ratio_
print(f"Variância explicada pelos componentes principais: {explained_variance}")
print("Variância explicada por 2 componentes:", sum(pca.explained_variance_ratio_))

<p align="justify">
Agora que projetamos os 64 atributos em dois componentes principais, podemos visualizá-lo na forma de um gráfico 2D. A célula abaixo cria uma visualização baseada em um gráfico de dispersão que apresenta o conjunto de dados após a projeção em dois componentes principais.
</p>

In [ ]:
# Visualização dos dados reduzidos
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.colorbar(label='Classe')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Dados projetados em 2 Componentes Principais')
plt.show()

<p align="justify">
Por fim, podemos utilizar o conjunto de dados processador para realizar o treinametno de um modelo de aprendizado simples a fim de verificar a acurácia resultante. No exemplo abaixo, iremos treinar um modelo de <strong>Random Forest</strong> e verificar sua acurácia utilizando a divisão em conjuntos de treino e teste realizada anteriormente.
</p>

In [ ]:
# Treinar um modelo Random Forest com dados reduzidos
model = RandomForestClassifier(random_state=42)
model.fit(X_train_pca, y_train)

# Fazer previsões e avaliar
y_pred = model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)

print(f"Acurácia do modelo com dados reduzidos pelo PCA: {accuracy:.2f}")